In [2]:
import pandas as pd

In [6]:
worker_order_df = pd.read_csv('workorderdata/cleanworkerorder.csv')


### Columns to encode
1. 'AffectedProduction'
2. ,'GrossProductionLoss',
3. 'Duration',
4. 'DaysFromLastFailure_int

In [ ]:
bins = [0, 10, 20, 30, 40]